# Value-at-Risk for Stocks: Boudoukh-Richardson-Whitelaw (BRW) Approach

### Lecture Notes by Jakov Ivan S. Dumbrique (jdumbrique@ateneo.edu)

MATH 100.2: Topics in Financial Mathematics II \
First Semester, S.Y. 2021-2022 \
Ateneo de Manila University

In [1]:
import numpy as np # Numerical Computing
import pandas as pd # Data wrangling

## Introduction to BRW Approach
The historical simulation approach assumes that historical returns are uniformly distributed and so quantiles are estimated based on the empirical distribution of returns.

Limitations of the historical simulation approach:

1. Extreme percentiles of the distribution are difficult to estimate with a small amount of data.
2. Returns are assumed to be i.i.d. and hence does not allow for time-varying volatility.

In an effort to address these limitations, we may extend the historical window to include more observations. However, longer historical windows diminish the weight assigned to more recent market phenomenon.

The BRW/Hybrid approach entails the use of exponentially-declining weights on historical data to estimate percentiles, with bigger weights assigned to more recent data.

The empirical distribution of historical returns now rely on the assigned weights. Quantiles are then calculated based on the cumulative weights after ordering historical returns in increasing order.

## Step 1: Assignment of Weights 

Let $\{R_0,R_1,\dots,R_{M-1}\}$ denote the observed one-day log-returns on the stock or portfolio, where $R_i$ is the one-day log-return observed $i$ days ago.
	
Let $0<\lambda<1$ be constant and let $w_i$ be the weight assigned to $R_i$, $i=0,1,\dots,M-1$. The weights decay exponentially, so we assume that $w_{i+1}=\lambda w_i$. To ensure that the weights sum to 1, we have
	\begin{equation}
	w_i = \left(\frac{1-\lambda}{1-\lambda^M}\right)\lambda^i, \qquad i=0,1,\dots,M-1.
	\end{equation}

Consider the single-asset case. Note that each historical return can be mapped to a scenario for the change in portfolio value $$\Delta P^{(i)} \approx NS_0 R_i,$$ where $N$ is the number of shares of the stock, $S_0$ is the current price of the stock per share, and $R_i$ is the one-day log-return observed $i$ days ago.
	
The weights $w_0, w_1,\dots w_{M-1}$ therefore also correspond to the collection $\Delta P^{(0)}, \Delta P^{(1)},\dots\Delta P^{(M-1)}$, the scenario space for the random variable $\Delta P$.


In [2]:
# These are from our programming session on HS
def get_return(df, d):
    """
    df is the original df
    
    appends returns series to df
    """
    df["previous"] = df["close"].shift(-d)
    df["return"] = np.log(df["close"]/df["previous"])
    return df 

In [3]:
def get_change_in_value_df(df, N):
    """
    df is the output of get_returns_df 
    
    appends change in portfolio value series to df
    """
    S0 = df.loc[0, "close"]
    df["change_in_value"] = N * S0 * df["return"]
    return df

In [4]:
def get_kth_percentile_discrete(df, d, alpha):
    """
    this returns the (1-alpha)th percentile of the ordered array of historical changes in portfolio values
    """
    M = len(df)-d
    k = int(np.floor((1-alpha)*M))
    var = abs(df["change_in_value"].sort_values(ignore_index=True)[k-1])
    
    return var

\begin{equation}
	w_i = \left(\frac{1-\lambda}{1-\lambda^M}\right)\lambda^i, \qquad i=0,1,\dots,M-1.
	\end{equation}

In [5]:
# This is from our programming session on EWMA 
# with a minor edit on ewma_par (now decay_par) and weight_lst formula
def get_weights_df(df, d, decay_par):
    """ 
    appends weights series to df
    """
    count_returns = len(df["close"])-d # M
    weight_lst = [(1-decay_par)*(decay_par**j)/(1-decay_par**count_returns) for j in range(count_returns)]
    df["weight"] = pd.Series(weight_lst)
    return df

<h3 style='color:green'> Example on Single-Asset Portfolio</h3> 
Today is February 23, 2018. You are a portfolio risk manager who is assigned to analyze the market risk of a portfolio of 700 PLDT (TEL) shares. Determine the portfolio's one-day 99% VaR using BRW approach with decay parameter of 0.76.
 


In [6]:
df = pd.read_csv("https://raw.githubusercontent.com/ateneomathdept/math100.2_2021Sem1/main/data/lectures/TEL_2018.csv")

In [7]:
df

,dt,close
0,2/23/18,1488.74
1,2/22/18,1510.86
2,2/21/18,1513.72
3,2/20/18,1536.65
4,2/16/18,1476.37
...,...,...
243,3/2/17,1430.29
244,3/1/17,1408.94
245,2/28/17,1394.35
246,2/27/17,1374.93


In [8]:
d = 1
N = 700
# step 1.1: generate your historical returns
df = get_return(df, d)
# Step 1.2: get your historical changes in portfolio values
df = get_change_in_value_df(df, N)
df

,dt,close,previous,return,change_in_value
0,2/23/18,1488.74,1510.86,-0.014749,-15370.094698
1,2/22/18,1510.86,1513.72,-0.001891,-1970.824622
2,2/21/18,1513.72,1536.65,-0.015035,-15667.749059
3,2/20/18,1536.65,1476.37,0.040018,41703.842890
4,2/16/18,1476.37,1475.71,0.000447,465.975103
...,...,...,...,...,...
243,3/2/17,1430.29,1408.94,0.015040,15673.009003
244,3/1/17,1408.94,1394.35,0.010409,10847.710303
245,2/28/17,1394.35,1374.93,0.014026,14616.263663
246,2/27/17,1374.93,1367.68,0.005287,5509.622658


In [9]:
decay_par = 0.76
# Step 1.3: generate the weights
df = get_weights_df(df, d, decay_par)
df

,dt,close,previous,return,change_in_value,weight
0,2/23/18,1488.74,1510.86,-0.014749,-15370.094698,2.400000e-01
1,2/22/18,1510.86,1513.72,-0.001891,-1970.824622,1.824000e-01
2,2/21/18,1513.72,1536.65,-0.015035,-15667.749059,1.386240e-01
3,2/20/18,1536.65,1476.37,0.040018,41703.842890,1.053542e-01
4,2/16/18,1476.37,1475.71,0.000447,465.975103,8.006922e-02
...,...,...,...,...,...,...
243,3/2/17,1430.29,1408.94,0.015040,15673.009003,2.617666e-30
244,3/1/17,1408.94,1394.35,0.010409,10847.710303,1.989426e-30
245,2/28/17,1394.35,1374.93,0.014026,14616.263663,1.511964e-30
246,2/27/17,1374.93,1367.68,0.005287,5509.622658,1.149092e-30


## Step 2: Constructing the Empirical Distribution

Arrange the $\Delta P$ scenario space in increasing order. 
	
The empirical cumulative distribution at $\Delta\tilde{P}_{(j)}$ is determined by the accumulation of weights $\psi_j$ defined as
	\begin{equation}
	\psi_j = \sum_{i=0}^j w_{(i)}, \qquad j=0,1,\dots,M-1.
	\end{equation}

In [10]:
df.sort_values(by="change_in_value")

,dt,close,previous,return,change_in_value,weight
68,11/13/17,1612.93,1740.89,-0.076344,-79559.535694,1.885973e-09
43,12/19/17,1477.31,1565.96,-0.058276,-60730.664517,1.799682e-06
10,2/8/18,1442.38,1516.47,-0.050091,-52200.460251,1.542933e-02
161,6/29/17,1713.94,1798.89,-0.048375,-50412.452299,1.553101e-20
49,12/11/17,1442.18,1509.60,-0.045689,-47613.180665,3.467985e-07
...,...,...,...,...,...,...
185,5/25/17,1666.95,1590.54,0.046922,48898.290082,2.141538e-23
198,5/5/17,1741.91,1661.55,0.047231,49220.599037,6.043695e-25
239,3/8/17,1532.88,1433.24,0.067211,70041.483709,7.846208e-30
19,1/26/18,1614.12,1489.54,0.080323,83705.595030,1.305177e-03


In [11]:
def get_ecdf(df):
    """
        assumes the input df have already passed through get_change_in_value_df and get_weights_df
    """
    df = df.sort_values(by="change_in_value")
    df["ecdf"] = df["weight"].cumsum()
    
    return df

In [12]:
df = get_ecdf(df)
df

,dt,close,previous,return,change_in_value,weight,ecdf
68,11/13/17,1612.93,1740.89,-0.076344,-79559.535694,1.885973e-09,1.885973e-09
43,12/19/17,1477.31,1565.96,-0.058276,-60730.664517,1.799682e-06,1.801568e-06
10,2/8/18,1442.38,1516.47,-0.050091,-52200.460251,1.542933e-02,1.543114e-02
161,6/29/17,1713.94,1798.89,-0.048375,-50412.452299,1.553101e-20,1.543114e-02
49,12/11/17,1442.18,1509.60,-0.045689,-47613.180665,3.467985e-07,1.543148e-02
...,...,...,...,...,...,...,...
185,5/25/17,1666.95,1590.54,0.046922,48898.290082,2.141538e-23,9.986948e-01
198,5/5/17,1741.91,1661.55,0.047231,49220.599037,6.043695e-25,9.986948e-01
239,3/8/17,1532.88,1433.24,0.067211,70041.483709,7.846208e-30,9.986948e-01
19,1/26/18,1614.12,1489.54,0.080323,83705.595030,1.305177e-03,1.000000e+00


## Step 3: Calculating the VaR

The $100\alpha\%$ value-at-risk is obtained by first identifying successive ordered pairs $(\Delta\tilde{P}^{(k)},\psi_k)$ and $(\Delta\tilde{P}^{(k+1)},\psi_{k+1})$ such that
	\begin{equation}
	\psi_k < 1-\alpha < \psi_{k+1}.
	\end{equation}
	(This is because $1-\alpha$ may not be a value assumed in the sequence $\psi_0,\psi_1,\dots,\psi_{M-1}$.)
	
	
Let $V$ be the linearly interpolated value of $\Delta P$ from the ordered pairs $(\Delta\tilde{P}^{(k)},\psi_k)$ and $(\Delta\tilde{P}^{(k+1)},\psi_{k+1})$ corresponding to the value $\psi=1-\alpha$. **The $100\alpha\%$ value-at-risk is therefore given by $|V|$.**

Step 1: Identify the successive ordered pairs $(\Delta\tilde{P}^{(k)},\psi_k)$ and $(\Delta\tilde{P}^{(k+1)},\psi_{k+1})$ such that
	\begin{equation}
	\psi_k < 1-\alpha < \psi_{k+1}.
	\end{equation}
Step 2: implement linear interpolation on $\Delta P$ from the ordered pairs $(\Delta\tilde{P}^{(k)},\psi_k)$ and $(\Delta\tilde{P}^{(k+1)},\psi_{k+1})$ corresponding to the value $\psi=1-\alpha$


Tip: use [np.interp()](https://numpy.org/doc/stable/reference/generated/numpy.interp.html) 

In [13]:
alpha = 0.99
var = abs(np.interp(1-alpha, df["ecdf"], df["change_in_value"]))
var

55203.09747955038

In [14]:
def d_day_alpha_percent_VaR_single_stock_brw(
    df, N, d, alpha, decay_par
):
    """Returns the d-day 100*alpha% VaR of a single stock using BRW Approach.
    
    Parameters
    ----------
    df : pandas.DataFrame
        has two columns: (1) dt [str] and (2) closing price [float]
        assumes the dates are arranged from newest to oldest, and the date today is the date on the first row  
    N : int
        number of shares for the sole stock
    d : int
        the value to be used in calculating the d-day VaR (e.g. 1-day, 5-day)
    alpha : float
        the value to be used in calculting the 100(alpha)% VaR (e.g. 0.99, 0.95)
    decay_par : float
        the value of the BRW decay parameter
    
    Returns
    -------
    float (2 decimal places)
       d-day 100*alpha% VaR of a single stock using BRW Approach.
    """
    # STEP 1: ASSIGNMENT OF WEIGHTS
    # step 1.1: generate your historical returns
    df = get_return(df, d)
    # Step 1.2: get your historical changes in portfolio values
    df = get_change_in_value_df(df, N)
    # Step 1.3: generate the weights
    df = get_weights_df(df, d, decay_par)
    
    # STEP 2: CONSTRUCT THE ECDF
    df = get_ecdf(df)
    
    # STEP 3: CALCULATE THE VAR
    var = abs(np.interp(1-alpha, df["ecdf"], df["change_in_value"]))
    
    return round(var, 2)

In [15]:
TEL_df = pd.read_csv("https://raw.githubusercontent.com/ateneomathdept/math100.2_2021Sem1/main/data/lectures/TEL_2018.csv")

In [16]:
d_day_alpha_percent_VaR_single_stock_brw(df=TEL_df, N=700, d=1, alpha=0.99, decay_par=0.76)

55203.1

## The BRW Approach: N-Asset Case

The BRW approach can be extended to the $N$-asset case by taking the $R_i$'s as the one-day log-return on the **entire portfolio**. To $R_i$ we assign the weight $w_i$ (as defined before).

If $P_0$ is the current value of the portfolio, then scenarios for the future one-day change in portfolio value $\Delta P$ can be obtained using the equation
	\begin{equation}
	\Delta P_i = P_0(e^{R_i}-1).
	\end{equation}

We then resume the BRW approach at Step 2 (Constructing the Empirical Distribution) using $\{\Delta P_i\}$ as the $\Delta P$ scenarios.
